In [5]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
import imageio
import os
from scipy import misc
import json
import pprint
import time

%matplotlib inline

In [6]:
BODY_PARTS = [
    "Nose",
    "Neck",
    "RShoulder",
    "RElbow",
    "RWrist",
    "LShoulder",
    "LElbow",
    "LWrist",
    "RHip",
    "RKnee",# Add 10 to remove blown-out artifacts
    "RAnkle",
    "LHip",
    "LKnee",
    "LAnkle",
    "REye",
    "LEye",
    "REar",
    "LEar",
    "Bkg",
]

In [11]:
OPENPOSE_ROOT = '/home/albert/openpose/'

POSE_ID_ROOT = '/home/albert/openpose/albert/pose_id/'
# POSE_ID_ROOT = '/Users/tradergllc/openpose/pose_id/'

In [12]:
def render_frame(f):
    img_file = POSE_ID_ROOT + 'DJI_seq/DJI_0005_%d.png' % f
    pose_file = POSE_ID_ROOT + 'DJI_pose/DJI_0005_%d_keypoints.json' % f
    render_file = POSE_ID_ROOT + 'DJI_render/DJI_0005_%d_rendered.png' % f
    #     heatmap_file = OPENPOSE_ROOT + 'DJI_seq/DJI_0005_' + f + '.png'

    img = misc.imread(img_file)
    render = misc.imread(render_file)

    with open(pose_file) as data_file:    
        data = json.load(data_file)

    for p in range(len(data['people'])):
        x_max, x_min = 0, img.shape[1]
        y_max, y_min = 0, img.shape[0]
        
        for k_type in data['people'][p].keys():
            pose_dict = {'x' : [], 'y' : [], 'c' : []}

            for i in range(0, len(data['people'][p][k_type]), 3):
                x = data['people'][p][k_type][i]
                y = data['people'][p][k_type][i + 1]
                c = data['people'][p][k_type][i + 2]
                
                img = cv2.circle(img,(int(x),int(y)), 5, (0,int(255 * c),int(255 * c)), -1)
                
                pose_dict['x'].append(x)
                pose_dict['y'].append(y)
                pose_dict['c'].append(c)
                
                if k_type == 'pose_keypoints':
                    if x < x_min and x > 0:
                        x_min = x
                    if x > x_max:
                        x_max = x
                    if y < y_min and y > 0:
                        y_min = y
                    if y > y_max:
                        y_max = y

            data['people'][p][k_type] = pose_dict
            
            diff_x = x_max - x_min
            diff_y = y_max - y_min
            r_x = 0.5
            r_y = 0.2
            img = cv2.rectangle(img, (int(x_min-r_x*diff_x),int(y_min-r_y*diff_y)), 
                                    (int(x_max+r_x*diff_x),int(y_max+r_y*diff_y)), 
                                    (0,255,0), 3)
    return img, len(data['people'])

In [13]:
!pwd

/home/albert/openpose/albert/pose_id


In [15]:
frames= []
f_n = 150
start = time.time()

for f in range(0, f_n * 5, 5):
    if f % 50 == 0:
        end = time.time()
        print("Frames rendered: %d, Time: %.4f" % (f / 5, end - start))

    img, n_people = render_frame(f)
    frames.append(img)
    
print("Render complete!")

Frames rendered: 0, Time: 0.0001
Frames rendered: 10, Time: 1.1538
Frames rendered: 20, Time: 2.3571
Frames rendered: 30, Time: 3.5483
Frames rendered: 40, Time: 4.7298
Frames rendered: 50, Time: 5.8969
Frames rendered: 60, Time: 7.1066
Frames rendered: 70, Time: 8.6141
Frames rendered: 80, Time: 10.5339
Frames rendered: 90, Time: 11.9814
Frames rendered: 100, Time: 13.3559
Frames rendered: 110, Time: 14.7130
Frames rendered: 120, Time: 16.2125
Frames rendered: 130, Time: 17.6067
Frames rendered: 140, Time: 18.9811
Render complete!


In [17]:
writer = imageio.get_writer('DJI_render.mp4', fps=5)

start = time.time()

for f in range(len(frames)):
    if f % 10 == 0:
        end = time.time()
        print("Frames rendered: %d, Time: %.4f" % (f, end - start))
    writer.append_data(frames[f])
writer.close()
print("Video render complete")

Frames rendered: 0, Time: 0.0005
Frames rendered: 10, Time: 0.4645
Frames rendered: 20, Time: 0.7007
Frames rendered: 30, Time: 0.9143
Frames rendered: 40, Time: 1.1449
Frames rendered: 50, Time: 2.2159
Frames rendered: 60, Time: 3.1080
Frames rendered: 70, Time: 3.7761
Frames rendered: 80, Time: 4.7524
Frames rendered: 90, Time: 5.7197
Frames rendered: 100, Time: 6.7263
Frames rendered: 110, Time: 7.6999
Frames rendered: 120, Time: 8.6390
Frames rendered: 130, Time: 9.5626
Frames rendered: 140, Time: 10.5452
Video render complete
